# Exploratory Data Analysis (EDA): Financial Transactions Dataset

**Dataset:** HuggingFace Financial_Transactions  
**Focus:** MCC 5411 - Grocery Stores and Supermarkets  
**Objective:** Analyze transaction patterns to prepare data for predictive modeling

---

## Notebook Sections:
1. Data Loading and Initial Inspection
2. Data Cleaning and Quality Checks
3. Weekly Aggregation and Descriptive Statistics
4. Feature Engineering (91 features)
5. Final Dataset Preparation

## 1. Data Loading and Initial Inspection

In [ ]:
import huggingface_hub as hf_hub

hf_hub.login(token="")

In [ ]:
from datasets import load_dataset

# Load Financial Transactions dataset from HuggingFace
ds = load_dataset("thiru1711/Financial_Transactions")

# Display sample record
ds['train'][0]

In [ ]:
import pandas as pd
import numpy as np

# Convert to pandas DataFrame
df = ds['train'].to_pandas()

# Display first few rows
df.head()

In [ ]:
# Select relevant columns for analysis
df_working = df[['transaction_id','date','card_brand','merchant_id', 'mcc','mcc_description', 'amount']]

# Display dataset info
df_working.info()

In [ ]:
# Analyze MCC distribution to identify top categories
mcc_counts = df_working['mcc'].value_counts()
print("Top 10 Most Frequent MCCs:")
print(mcc_counts.head(10))

# Include MCC descriptions
print("\n\nTop 10 MCCs with Descriptions:")
top_mccs = df_working.groupby(['mcc', 'mcc_description']).size().reset_index(name='count')
top_mccs = top_mccs.sort_values('count', ascending=False).head(10)
top_mccs

## 2. Focus on MCC 5411: Grocery Stores and Supermarkets

In [ ]:
# Filter for MCC 5411 only (Grocery Stores, Supermarkets)
gs_df = df_working[df_working['mcc'] == '5411'].copy()

# Convert date column to datetime
gs_df['date'] = pd.to_datetime(gs_df['date'])

gs_df.info()

## 3. Weekly Aggregation and Descriptive Statistics

In [ ]:
# Create a week column for grouping
gs_df['week'] = gs_df['date'].dt.to_period('W')

# Group by week and calculate metrics
weekly_stats = gs_df.groupby('week').agg({
    'transaction_id': 'count',  # Count of transactions
    'amount': ['sum', 'mean']   # Total and average amount
})

# Flatten column names
weekly_stats.columns = ['transaction_count', 'total_amount', 'avg_amount']
weekly_stats.index = weekly_stats.index.to_timestamp()

# Round numeric columns
weekly_stats['total_amount'] = weekly_stats['total_amount'].round(2)
weekly_stats['avg_amount'] = weekly_stats['avg_amount'].round(2)

print(f"Weekly dataset created: {len(weekly_stats)} weeks")
print(f"Date range: {weekly_stats.index.min()} to {weekly_stats.index.max()}")
weekly_stats.head()

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3,1, figsize=(15,10))

# Plot transaction count
axes[0].plot(weekly_stats.index, weekly_stats['transaction_count'], linewidth=1.5)
axes[0].set_ylabel('Total Transactions', fontsize=11)
axes[0].set_title('Transaction Count per Week', fontsize=12)
axes[0].grid(True, alpha=0.3)

# Plot total amount
axes[1].plot(weekly_stats.index, weekly_stats['total_amount'], linewidth=1.5, color='green')
axes[1].set_ylabel('Total Amount ($)', fontsize=11)
axes[1].set_title('Total Amount per Week', fontsize=12)
axes[1].grid(True, alpha=0.3)

# Plot average amount
axes[2].plot(weekly_stats.index, weekly_stats['avg_amount'], linewidth=1.5, color='orange')
axes[2].set_ylabel('Average Amount ($)', fontsize=11)
axes[2].set_xlabel('Week', fontsize=11)
axes[2].set_title('Average Transaction Amount per Week', fontsize=12)
axes[2].grid(True, alpha=0.3)

plt.suptitle('MCC 5411: Grocery Stores & Supermarkets - Weekly Metrics', fontsize=14, y=0.995)
plt.tight_layout()
plt.show()

In [ ]:
# Comprehensive descriptive statistics
weekly_stats.describe()

In [ ]:
# Detailed statistical analysis
print("="*80)
print("WEEKLY STATISTICS - Detailed Analysis")
print("="*80)
print(f"\nDataset Size: {len(weekly_stats)} weeks")
print(f"Date Range: {weekly_stats.index.min().strftime('%Y-%m-%d')} to {weekly_stats.index.max().strftime('%Y-%m-%d')}\n")

for col in ['transaction_count', 'total_amount', 'avg_amount']:
    print(f"\n{col.upper().replace('_', ' ')}:")
    mean_val = weekly_stats[col].mean()
    std_val = weekly_stats[col].std()
    min_val = weekly_stats[col].min()
    max_val = weekly_stats[col].max()
    cv = (std_val / mean_val) * 100  # Coefficient of variation (%)
    
    print(f"  Mean: {mean_val:,.2f}")
    print(f"  Std Dev: {std_val:,.2f}")
    print(f"  CV (Coefficient of Variation): {cv:.2f}%")
    print(f"  Range: {min_val:,.2f} to {max_val:,.2f}")

# Growth metrics
print("\n" + "="*80)
print("GROWTH METRICS")
print("="*80)

for col in ['transaction_count', 'total_amount', 'avg_amount']:
    first_val = weekly_stats[col].iloc[0]
    last_val = weekly_stats[col].iloc[-1]
    total_growth = ((last_val - first_val) / first_val) * 100
    weekly_avg_growth = total_growth / len(weekly_stats)
    
    print(f"\n{col.upper().replace('_', ' ')}:")
    print(f"  First value: {first_val:,.2f}")
    print(f"  Last value: {last_val:,.2f}")
    print(f"  Total growth: {total_growth:.2f}%")
    print(f"  Average weekly growth: {weekly_avg_growth:.2f}%")

## 4. Data Quality Checks: Identifying Incomplete Weeks

In [ ]:
# Check for incomplete weeks (first and last weeks often incomplete)
mean_count = weekly_stats['transaction_count'].mean()
first_count = weekly_stats['transaction_count'].iloc[0]
last_count = weekly_stats['transaction_count'].iloc[-1]

# Set threshold at 70% of mean weekly transactions
threshold = 0.7 * mean_count

print("="*80)
print("DATA QUALITY CHECK: Identifying Incomplete Weeks")
print("="*80)
print(f"\nMean weekly transaction count: {mean_count:,.0f}")
print(f"Threshold (70% of mean): {threshold:,.0f}\n")

print("FIRST WEEK:")
print(f"  Week: {weekly_stats.index[0].strftime('%Y-%m-%d')}")
print(f"  Transaction count: {first_count:,.0f}")
print(f"  Percentage of mean: {(first_count/mean_count)*100:.1f}%")
remove_first = first_count < threshold
print(f"  Assessment: {'❌ INCOMPLETE - REMOVE' if remove_first else '✅ Complete'}")

print("\nLAST WEEK:")
print(f"  Week: {weekly_stats.index[-1].strftime('%Y-%m-%d')}")
print(f"  Transaction count: {last_count:,.0f}")
print(f"  Percentage of mean: {(last_count/mean_count)*100:.1f}%")
remove_last = last_count < threshold
print(f"  Assessment: {'❌ INCOMPLETE - REMOVE' if remove_last else '✅ Complete'}")

In [ ]:
# Remove incomplete weeks if necessary
weekly_clean = weekly_stats.copy()

if remove_first:
    weekly_clean = weekly_clean.iloc[1:]
    print(f"✅ Removed first week: {weekly_stats.index[0].strftime('%Y-%m-%d')}")

if remove_last:
    weekly_clean = weekly_clean.iloc[:-1]
    print(f"✅ Removed last week: {weekly_stats.index[-1].strftime('%Y-%m-%d')}")

print("\n" + "="*80)
print("CLEANED DATASET")
print("="*80)
print(f"Original weeks: {len(weekly_stats)}")
print(f"Cleaned weeks: {len(weekly_clean)}")
print(f"Weeks removed: {len(weekly_stats) - len(weekly_clean)}")
print(f"\nNew date range: {weekly_clean.index.min().strftime('%Y-%m-%d')} to {weekly_clean.index.max().strftime('%Y-%m-%d')}")

weekly_clean.head()

## 5. Feature Engineering: Creating 91 Predictive Features

We'll create comprehensive features to capture temporal patterns, trends, and seasonality without using external data sources.

In [ ]:
# Create feature-engineered dataset
features_df = weekly_clean.copy()

# 1. TEMPORAL FEATURES
features_df['week_of_year'] = features_df.index.isocalendar().week
features_df['month'] = features_df.index.month
features_df['quarter'] = features_df.index.quarter
features_df['year'] = features_df.index.year
features_df['day_of_year'] = features_df.index.dayofyear

# 2. CYCLICAL ENCODING (sine/cosine to preserve cyclical nature)
features_df['week_sin'] = np.sin(2 * np.pi * features_df['week_of_year'] / 52)
features_df['week_cos'] = np.cos(2 * np.pi * features_df['week_of_year'] / 52)
features_df['month_sin'] = np.sin(2 * np.pi * features_df['month'] / 12)
features_df['month_cos'] = np.cos(2 * np.pi * features_df['month'] / 12)

# 3. TREND FEATURE
features_df['time_index'] = np.arange(len(features_df))

print("="*80)
print("TEMPORAL FEATURES ADDED")
print("="*80)
print("✅ Basic temporal: week_of_year, month, quarter, year, day_of_year")
print("✅ Cyclical encoding: week_sin/cos, month_sin/cos (preserves seasonality)")
print("✅ Trend: time_index (linear progression)")
print(f"\nTotal temporal features: 10")

features_df.head()

In [ ]:
# 4. LAG FEATURES (previous week's values)
lag_periods = [1, 2, 3, 4, 8, 12, 52]  # 1w, 2w, 3w, 1m, 2m, 3m, 1y

for lag in lag_periods:
    for col in ['transaction_count', 'total_amount', 'avg_amount']:
        features_df[f'{col}_lag{lag}'] = features_df[col].shift(lag)

print("="*80)
print("LAG FEATURES ADDED")
print("="*80)
print(f"Lag periods: {lag_periods} weeks")
print(f"For metrics: transaction_count, total_amount, avg_amount")
print(f"\nTotal lag features: {len(lag_periods) * 3} = 21")
print(f"\nExamples:")
print(f"  - transaction_count_lag1 = transaction count from 1 week ago")
print(f"  - transaction_count_lag52 = transaction count from same week last year")

# Show sample lag features
features_df.iloc[52:55][['transaction_count', 'transaction_count_lag1', 'transaction_count_lag4', 'transaction_count_lag52']].head()

In [ ]:
# 5. ROLLING WINDOW FEATURES (moving averages and statistics)
rolling_windows = [4, 8, 12, 26]  # 1 month, 2 months, 3 months, 6 months

for window in rolling_windows:
    for col in ['transaction_count', 'total_amount', 'avg_amount']:
        # Rolling statistics using shift(1) to prevent data leakage
        features_df[f'{col}_ma{window}'] = features_df[col].shift(1).rolling(window=window).mean()
        features_df[f'{col}_std{window}'] = features_df[col].shift(1).rolling(window=window).std()
        features_df[f'{col}_min{window}'] = features_df[col].shift(1).rolling(window=window).min()
        features_df[f'{col}_max{window}'] = features_df[col].shift(1).rolling(window=window).max()

print("="*80)
print("ROLLING WINDOW FEATURES ADDED")
print("="*80)
print(f"Windows: {rolling_windows} weeks")
print(f"Statistics: Moving Average, Std Dev, Min, Max")
print(f"For metrics: transaction_count, total_amount, avg_amount")
print(f"\nTotal rolling features: {len(rolling_windows) * 3 * 4} = 48")
print(f"\n⚠️  All rolling features use shift(1) to prevent data leakage")
print(f"   (only past data used, not current week)")

# Show sample rolling features
features_df.iloc[50:55][['transaction_count', 'transaction_count_ma4', 'transaction_count_std4', 'transaction_count_ma26']].head()

In [ ]:
# 6. DIFFERENCE FEATURES (changes and growth rates)
for col in ['transaction_count', 'total_amount', 'avg_amount']:
    # Week-over-week change
    features_df[f'{col}_diff1'] = features_df[col].diff(1)
    features_df[f'{col}_pct_change1'] = features_df[col].pct_change(1)
    
    # Year-over-year change (52 weeks)
    features_df[f'{col}_diff52'] = features_df[col].diff(52)
    features_df[f'{col}_pct_change52'] = features_df[col].pct_change(52)

print("="*80)
print("DIFFERENCE FEATURES ADDED")
print("="*80)
print("For metrics: transaction_count, total_amount, avg_amount")
print("\nFeatures created:")
print("  - diff1: Week-over-week absolute change")
print("  - pct_change1: Week-over-week % change")
print("  - diff52: Year-over-year absolute change")
print("  - pct_change52: Year-over-year % change")
print(f"\nTotal difference features: {3 * 4} = 12")

# Show sample difference features
features_df.iloc[52:57][['transaction_count', 'transaction_count_diff1', 'transaction_count_pct_change1', 'transaction_count_diff52']].head()

## 6. Feature Summary and Final Dataset Preparation

In [ ]:
# Complete feature summary
print("="*80)
print("COMPLETE FEATURE ENGINEERING SUMMARY")
print("="*80)

print(f"\n📊 Original Metrics: 3")
print(f"   - transaction_count, total_amount, avg_amount")

print(f"\n📅 Temporal Features: 10")
print(f"   - Basic: week_of_year, month, quarter, year, day_of_year")
print(f"   - Cyclical: week_sin, week_cos, month_sin, month_cos")
print(f"   - Trend: time_index")

print(f"\n⏮️  Lag Features: 21")
print(f"   - Lags: {lag_periods}")
print(f"   - For each of 3 metrics")

print(f"\n📈 Rolling Features: 48")
print(f"   - Windows: {rolling_windows}")
print(f"   - Stats: mean, std, min, max")
print(f"   - For each of 3 metrics")

print(f"\n🔄 Difference Features: 12")
print(f"   - Absolute and % change")
print(f"   - 1-week and 52-week differences")
print(f"   - For each of 3 metrics")

print(f"\n{'='*80}")
print(f"TOTAL FEATURES IN DATASET: {features_df.shape[1]}")
print(f"  - 3 target metrics")
print(f"  - {features_df.shape[1] - 3} predictor features")
print(f"{'='*80}")

print(f"\n📋 Dataset Status:")
print(f"   Total rows: {len(features_df)}")
print(f"   Rows with NaN: {features_df.isnull().any(axis=1).sum()}")
print(f"   Complete cases: {features_df.dropna().shape[0]}")

In [ ]:
# Remove rows with NaN values (due to lags and rolling windows)
features_clean = features_df.dropna().copy()

print("="*80)
print("FINAL CLEAN DATASET FOR MODELING")
print("="*80)
print(f"\n✅ Cleaned dataset: {len(features_clean)} weeks")
print(f"   Removed {len(features_df) - len(features_clean)} weeks with NaN values")
print(f"\n📅 Date range:")
print(f"   Start: {features_clean.index.min().strftime('%Y-%m-%d')}")
print(f"   End: {features_clean.index.max().strftime('%Y-%m-%d')}")
print(f"   Duration: ~{len(features_clean) / 52:.1f} years")

print(f"\n📊 Dataset dimensions:")
print(f"   Rows (weeks): {features_clean.shape[0]}")
print(f"   Columns (features): {features_clean.shape[1]}")

print(f"\n✅ Dataset ready for modeling!")

features_clean.info()

In [ ]:
# Save the cleaned dataset for modeling
features_clean.to_csv('grocery_store_features_clean.csv')
print("✅ Dataset saved to: grocery_store_features_clean.csv")

## 7. Quick Feature Correlation Analysis

In [ ]:
import seaborn as sns

# Select key features for correlation analysis (transaction_count example)
key_features = [
    'transaction_count',
    'transaction_count_lag1', 'transaction_count_lag4', 'transaction_count_lag52',
    'transaction_count_ma4', 'transaction_count_ma12',
    'transaction_count_diff1', 'transaction_count_pct_change52',
    'week_of_year', 'month', 'time_index',
    'week_sin', 'week_cos'
]

# Calculate correlations
corr_matrix = features_clean[key_features].corr()

# Plot correlation heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Key Features vs Transaction Count', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

# Print strongest correlations
print("="*80)
print("STRONGEST CORRELATIONS WITH TRANSACTION_COUNT")
print("="*80)
corr_with_target = corr_matrix['transaction_count'].abs().sort_values(ascending=False)
print(corr_with_target.head(15))

---

## ✅ EDA Complete!

**Summary:**
- ✅ Loaded Financial Transactions dataset from HuggingFace
- ✅ Filtered for MCC 5411 (Grocery Stores)
- ✅ Created weekly aggregations (512 weeks → 512 clean weeks)
- ✅ Identified and removed incomplete weeks
- ✅ Engineered 91 features (temporal, lag, rolling, difference)
- ✅ Final clean dataset: **460 weeks** with **94 columns**

**Next Steps:**
- Proceed to **02_Preliminary_Modelling.ipynb** for model training and evaluation